<a href="https://colab.research.google.com/github/Faheem2k19/Malaria-Detection/blob/master/malaria_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [7]:
train = pd.read_csv("C:/Users/fahim/OneDrive/Documents/GitHub/Malaria-Detection/datasets/train.csv")
test = pd.read_csv("C:/Users/fahim/OneDrive/Documents/GitHub/Malaria-Detection/datasets/test.csv")

In [8]:
train.head()

,label,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_2490,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499
0,Parasitized,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parasitized,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parasitized,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parasitized,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parasitized,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
train.tail()

,label,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_2490,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499
22041,Uninfected,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22042,Uninfected,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22043,Uninfected,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22044,Uninfected,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22045,Uninfected,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train.describe()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_2490,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499
count,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,...,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000,22046.000000
mean,0.082691,0.158396,0.263313,0.462624,0.699809,0.940533,1.256237,1.752245,2.414678,3.455502,...,2.935907,2.144289,1.475506,1.036242,0.735553,0.457543,0.264901,0.167559,0.089631,0.044861
std,3.222090,4.543716,6.065143,8.100107,10.053545,11.834619,13.789559,16.426943,19.113640,23.061772,...,21.399852,18.230696,15.027315,12.475082,10.453670,7.980879,5.996918,4.624739,3.298136,2.566040
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,174.000000,189.000000,197.000000,205.000000,205.000000,203.000000,203.000000,206.000000,209.000000,209.000000,...,207.000000,207.000000,207.000000,209.000000,205.000000,198.000000,191.000000,190.000000,186.000000,185.000000


In [9]:
X_train  = train.drop(['label'],axis=1).values
y_train  = train['label'].values

X_test  = test.drop(['label'],axis=1).values
y_test  = test['label'].values

In [10]:
X_test.shape, y_test.shape


((5512, 2500), (5512,))